# Importação

In [280]:
import os
import glob
import pandas as pd

from typing import List, Dict, TextIO, DefaultDict
from collections import defaultdict

import commands_dictionary

In [203]:
path_logs = r'/mnt/c/Users/mathe/Documents/AIML_logs/treinamento_aiml/logs'

In [204]:
logs = pd.DataFrame()

In [205]:
def get_equipment_type(path: str):
    """Returns a list with equipment's types.
    
    ## Args:
        path (str): Logs path.
        
    ## Returns:
        List (str): List with equipment's types.
    """
    
    equipment_types = []
    entries = os.listdir(path)
    for entry in entries:
        equipment_types.append(entry)
    return equipment_types

In [206]:
def get_run_ids(row, path: str) -> Dict[str, List[str]]:
    """Returns a dict with equipment's types as keys and a list of run_ids as values.

    ## Args:
        path (str): Logs path.
        
    ## Returns:
        Dict (str, List[str]): Dict with equipment's types as keys and a list of run_ids as values.
    """
    
    entries = os.listdir(f'{path}/{row["equipment_type"]}')
    for entry in entries:
        if entry == '220818_171756_851_8721_win':
            return entry

In [207]:
def get_test_case(row, path: str) -> Dict[str, List[str]]:
    """Returns a dict with run_ids as keys and a list of test_cases names as values.

    ## Args:
        path (str): Logs path.
        
    ## Returns:
        Dict (str, List[str]): Dict with run_ids as keys and a list of test_cases names as values.
    """
    
    test_cases = []
    files = next(os.walk(f'{path}/{row["equipment_type"]}/{row["run_id"]}'))[2]
    for file in files:
        if file.endswith('.txt', -4) and file != 'seed.txt' and file == 'test_case_6042.txt':
            test_cases.append(file[10:-4])
    return test_cases

In [208]:
def open_file(path: str, eq_type: str, run_id: str, test_case: str) -> TextIO:
    """Open a test_case_*.txt file by receiving the path of the equipments, equipment type, run_id 
        and the number of the test case as arguments. Returns the file.

    ## Args:
        path (str): path where each equipment is located.
        eq_type (str): equipment_type.
        run_id (str): run_id.
        test_case (str): number of the test_case file.
        
    ## Returns:
        file (TextIO): the opened test_case file.
    """
    
    file = open(f'{path}/{eq_type}/{run_id}/test_case_{test_case}.txt', 'r', encoding='cp860')
    return file

In [209]:
def get_timestamp(row, path: str):
    timestamps = []
    file = open_file(path, row['equipment_type'], row['run_id'], row['test_case'])
    log_lines = file.readlines()
    for line in log_lines:
        if 'OPTICALTX:' in line or 'TX:' in line:
            timestamps.append(line[1:24])
    file.close()
    return timestamps

In [210]:
def get_baud_rate(row, path: str):
    file = open_file(path, row['equipment_type'], row['run_id'], row['test_case'])
    log_lines = file.readlines()
    for line in log_lines:
        if row['timestamp'] == line[1:24] and ('OPTICALTX:' in line or 'TX:' in line):
            baud_rate = line[27:31]
            file.close()
            return baud_rate

In [211]:
def get_slot(row, path: str):
    file = open_file(path, row['equipment_type'], row['run_id'], row['test_case'])
    log_lines = file.readlines()
    for line in log_lines:
        if row['timestamp'] == line[1:24] and ('OPTICALTX:' in line or 'TX:' in line):
            slot = line[35:38].strip()
            file.close()
            return slot

In [212]:
def get_serial_port(row, path: str):
    file = open_file(path, row['equipment_type'], row['run_id'], row['test_case'])
    log_lines = file.readlines()
    for line in log_lines:
        if row['timestamp'] == line[1:24] and 'OPTICALTX:' in line:
            serial_port = line[40:49]
            file.close()
            return serial_port
        elif row['timestamp'] == line[1:24] and 'TX:' in line:
            serial_port = line[40:45]
            file.close()
            return serial_port

In [213]:
def get_tx_data(row, path: str):
    file = open_file(path, row['equipment_type'], row['run_id'], row['test_case'])
    log_lines = file.readlines()
    for line in log_lines:
        if row['timestamp'] == line[1:24] and 'OPTICALTX:' in line :
            tx_data = line[51:].rstrip('\n')
            file.close()
            return tx_data
        elif row['timestamp'] == line[1:24] and 'TX:' in line:
            tx_data = line[51:].rstrip('\n')
            file.close()
            return tx_data

In [214]:
# def get_rx_data(row, path: str):
#     file = open_file(path, row['equipment_type'], row['run_id'], row['test_case'])
#     log_lines = file.readlines()
#     for i, line in enumerate(log_lines):
#         if 'OPTICALRX:' in line:
#             tx_line = log_lines[i-1]
#             if tx_line[51:].rstrip('\n') == row['tx_data'] and tx_line[1:24] == row['timestamp']:
#                 rx_data = line[51:].rstrip('\n')
#                 file.close()
#                 return rx_data
#         elif 'RX:' in line:
#             tx_line = log_lines[i-1]
#             if tx_line[51:].rstrip('\n') == row['tx_data'] and tx_line[1:24] == row['timestamp']:
#                 rx_data = line[51:].rstrip('\n')
#                 file.close()
#                 return rx_data

In [215]:
def get_rx_data(row, path: str):
    file = open_file(path, row['equipment_type'], row['run_id'], row['test_case'])
    log_lines = file.readlines()
    for i, line in enumerate(log_lines):
        if 'TX:' in line and line[51:].rstrip('\n') == row['tx_data'] and line[1:24] == row['timestamp']:
            try:    
                while True:
                    j = 1
                    rx_line = log_lines[i+j]
                    if 'OPTICALRX:' in rx_line and row['serial_port'] == rx_line[40:49]:
                        rx_data = rx_line[51:].rstrip('\n')
                        file.close()
                        return rx_data
                    elif 'OPTICALTX:' in rx_line:
                        j += 1
                        continue
                    elif 'RX:' in rx_line and row['serial_port'] == rx_line[40:45]:
                        rx_data = rx_line[51:].rstrip('\n')
                        file.close()
                        return rx_data
                    elif 'TX:' in rx_line:
                        j += 1
                        continue
                    else:
                        return 'não encontrado'
            except IndexError:
                pass

In [216]:
def get_timestamp_rx(row, path: str):
    file = open_file(path, row['equipment_type'], row['run_id'], row['test_case'])
    log_lines = file.readlines()
    for i, line in enumerate(log_lines):
        if 'TX:' in line and line[51:].rstrip('\n') == row['tx_data'] and line[1:24] == row['timestamp']:
            try:    
                while True:
                    j = 1
                    rx_line = log_lines[i+j]
                    if 'OPTICALRX:' in rx_line and row['serial_port'] == rx_line[40:49]:
                        timestamp = rx_line[1:24].rstrip('\n')
                        file.close()
                        return timestamp
                    elif 'OPTICALTX:' in rx_line:
                        j += 1
                        continue
                    elif 'RX:' in rx_line and row['serial_port'] == rx_line[40:45]:
                        timestamp = rx_line[1:24].rstrip('\n')
                        file.close()
                        return timestamp
                    elif 'TX:' in rx_line:
                        j += 1
                        continue
                    else:
                        return 'não encontrado'
            except IndexError:
                pass

In [217]:
def get_response_time(row):
    if row['timestamp'] < row['timestamp_rx']:
        response_time = row['timestamp_rx'] - row['timestamp']
        return response_time
    else:
        response_time = row['timestamp'] - row['timestamp_rx']
        return response_time

In [218]:
def microseconds_to_seconds(row):
    response_time = (row['response_time'].microseconds) * 10e-4
    return response_time

In [273]:
def get_command_protocol(row):
    if row['tx_data'][:2] == '02' and row['tx_data'][-1:-3:-1] == '30':
        return 'Production'
    elif len(row['tx_data'].replace(' ', ''))/2 == 66 or len(row['rx_data'].replace(' ', ''))/2 == 258:
        return 'ABNT'
    else:
        return 'protocol not identified'

In [309]:
def get_command_name(row):
    try:    
        if row['command_protocol'] == 'Production':
            command = row['tx_data'][3:5].lower()
            protocol_command = commands_dictionary.production_commands.get(command)
            name = protocol_command.get('name')
            return name
        elif row['command_protocol'] == 'ABNT':
            command = row['tx_data'][:2].lower()
            protocol_command = commands_dictionary.abnt_commands.get(command)
            if protocol_command is None:
                protocol_command = commands_dictionary.abnt_extended_commands.get(command)
                name = protocol_command.get('name')
                return name
            else:
                return protocol_command.get('name')
    except AttributeError:
        return 'not found'

In [219]:
logs['equipment_type'] = get_equipment_type(path_logs)

In [220]:
logs['run_id'] = logs.apply(get_run_ids, path=path_logs, axis=1)

In [221]:
logs['test_case'] = logs.apply(get_test_case, path=path_logs, axis=1)
logs = logs.explode('test_case', ignore_index=True)

In [222]:
logs['timestamp'] = logs.apply(get_timestamp, path=path_logs, axis=1)
logs = logs.explode('timestamp', ignore_index=True)
logs

,equipment_type,run_id,test_case,timestamp
0,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.655
1,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.685
2,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.846
3,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:15.392
4,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:15.613
...,...,...,...,...
78,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:05.646
79,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:05.950
80,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:06.270
81,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:06.588


In [223]:
logs['baud_rate'] = logs.apply(get_baud_rate, path=path_logs, axis=1)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate
0,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.655,9600
1,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.685,9600
2,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.846,9600
3,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:15.392,9600
4,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:15.613,9600
...,...,...,...,...,...
78,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:05.646,9600
79,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:05.950,9600
80,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:06.270,9600
81,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:06.588,9600


In [224]:
logs['slot'] = logs.apply(get_slot, path=path_logs, axis=1)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot
0,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.655,9600,B
1,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.685,9600,B
2,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.846,9600,B
3,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:15.392,9600,B
4,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:15.613,9600,B
...,...,...,...,...,...,...
78,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:05.646,9600,OPT
79,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:05.950,9600,OPT
80,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:06.270,9600,OPT
81,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:06.588,9600,OPT


In [225]:
logs['serial_port'] = logs.apply(get_serial_port, path=path_logs, axis=1)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port
0,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.655,9600,B,COM22
1,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.685,9600,B,COM22
2,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.846,9600,B,COM22
3,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:15.392,9600,B,COM22
4,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:15.613,9600,B,COM22
...,...,...,...,...,...,...,...
78,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:05.646,9600,OPT,COM20
79,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:05.950,9600,OPT,COM20
80,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:06.270,9600,OPT,COM20
81,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:06.588,9600,OPT,COM20


In [226]:
logs['tx_data'] = logs.apply(get_tx_data, path=path_logs, axis=1)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data
0,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.655,9600,B,COM22,02 01 07 00 09 0C 03
1,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.685,9600,B,COM22,02 01 07 00 00 05 03
2,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.846,9600,B,COM22,02 01 07 01 08 0C 03
3,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:15.392,9600,B,COM22,02 5D FF 05 00 00 00 00 00 00 00 00 00 00 00 0...
4,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:15.613,9600,B,COM22,02 6A AD 09 00 01 00 10 00 00 00 00 00 00 00 0...
...,...,...,...,...,...,...,...,...
78,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:05.646,9600,OPT,COM20,06
79,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:05.950,9600,OPT,COM20,06
80,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:06.270,9600,OPT,COM20,06
81,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:06.588,9600,OPT,COM20,06


In [227]:
logs['rx_data'] = logs.apply(get_rx_data, path=path_logs, axis=1)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data
0,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.655,9600,B,COM22,02 01 07 00 09 0C 03,02 01 12 06 01 00 00 00 00 00 00 00 00 00 00 0...
1,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.685,9600,B,COM22,02 01 07 00 00 05 03,02 01 86 06 02 00 00 00 01 00 00 00 00 00 00 0...
2,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.846,9600,B,COM22,02 01 07 01 08 0C 03,02 01 DE 06 20 00 00 80 01 02 03 00 F2 3F 01 0...
3,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:15.392,9600,B,COM22,02 5D FF 05 00 00 00 00 00 00 00 00 00 00 00 0...,02 5D 06 06 5E 03
4,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:15.613,9600,B,COM22,02 6A AD 09 00 01 00 10 00 00 00 00 00 00 00 0...,02 6A 06 06 69 03
...,...,...,...,...,...,...,...,...,...
78,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:05.646,9600,OPT,COM20,06,26 12 34 56 78 00 58 47 00 10 00 00 48 10 00 0...
79,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:05.950,9600,OPT,COM20,06,26 12 34 56 78 00 59 10 00 00 7F 00 10 00 00 8...
80,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:06.270,9600,OPT,COM20,06,26 12 34 56 78 00 60 00 00 B6 10 00 00 B7 00 1...
81,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:06.588,9600,OPT,COM20,06,26 12 34 56 78 10 61 ED 00 10 00 00 EE 10 00 0...


In [228]:
logs['timestamp_rx'] = logs.apply(get_timestamp_rx, path=path_logs, axis=1)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,timestamp_rx
0,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.655,9600,B,COM22,02 01 07 00 09 0C 03,02 01 12 06 01 00 00 00 00 00 00 00 00 00 00 0...,19/08/2022 04:58:14.684
1,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.685,9600,B,COM22,02 01 07 00 00 05 03,02 01 86 06 02 00 00 00 01 00 00 00 00 00 00 0...,19/08/2022 04:58:14.844
2,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:14.846,9600,B,COM22,02 01 07 01 08 0C 03,02 01 DE 06 20 00 00 80 01 02 03 00 F2 3F 01 0...,19/08/2022 04:58:15.100
3,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:15.392,9600,B,COM22,02 5D FF 05 00 00 00 00 00 00 00 00 00 00 00 0...,02 5D 06 06 5E 03,19/08/2022 04:58:15.547
4,8721,220818_171756_851_8721_win,6042,19/08/2022 04:58:15.613,9600,B,COM22,02 6A AD 09 00 01 00 10 00 00 00 00 00 00 00 0...,02 6A 06 06 69 03,19/08/2022 04:58:15.739
...,...,...,...,...,...,...,...,...,...,...
78,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:05.646,9600,OPT,COM20,06,26 12 34 56 78 00 58 47 00 10 00 00 48 10 00 0...,19/08/2022 04:59:05.946
79,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:05.950,9600,OPT,COM20,06,26 12 34 56 78 00 59 10 00 00 7F 00 10 00 00 8...,19/08/2022 04:59:06.267
80,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:06.270,9600,OPT,COM20,06,26 12 34 56 78 00 60 00 00 B6 10 00 00 B7 00 1...,19/08/2022 04:59:06.584
81,8721,220818_171756_851_8721_win,6042,19/08/2022 04:59:06.588,9600,OPT,COM20,06,26 12 34 56 78 10 61 ED 00 10 00 00 EE 10 00 0...,19/08/2022 04:59:06.889


In [229]:
logs['timestamp'] = pd.to_datetime(logs['timestamp'], dayfirst=True)
logs['timestamp_rx'] = pd.to_datetime(logs['timestamp_rx'], dayfirst=True)
logs['response_time'] = logs.apply(get_response_time, axis=1)

In [230]:
logs['response_time'] = logs.apply(microseconds_to_seconds, axis=1)
logs

In [274]:
logs['command_protocol'] = logs.apply(get_command_protocol, axis=1)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,timestamp_rx,response_time,command_protocol
0,8721,220818_171756_851_8721_win,6042,2022-08-19 04:58:14.655,9600,B,COM22,02 01 07 00 09 0C 03,02 01 12 06 01 00 00 00 00 00 00 00 00 00 00 0...,2022-08-19 04:58:14.684,29.0,Production
1,8721,220818_171756_851_8721_win,6042,2022-08-19 04:58:14.685,9600,B,COM22,02 01 07 00 00 05 03,02 01 86 06 02 00 00 00 01 00 00 00 00 00 00 0...,2022-08-19 04:58:14.844,159.0,Production
2,8721,220818_171756_851_8721_win,6042,2022-08-19 04:58:14.846,9600,B,COM22,02 01 07 01 08 0C 03,02 01 DE 06 20 00 00 80 01 02 03 00 F2 3F 01 0...,2022-08-19 04:58:15.100,254.0,Production
3,8721,220818_171756_851_8721_win,6042,2022-08-19 04:58:15.392,9600,B,COM22,02 5D FF 05 00 00 00 00 00 00 00 00 00 00 00 0...,02 5D 06 06 5E 03,2022-08-19 04:58:15.547,155.0,Production
4,8721,220818_171756_851_8721_win,6042,2022-08-19 04:58:15.613,9600,B,COM22,02 6A AD 09 00 01 00 10 00 00 00 00 00 00 00 0...,02 6A 06 06 69 03,2022-08-19 04:58:15.739,126.0,Production
...,...,...,...,...,...,...,...,...,...,...,...,...
78,8721,220818_171756_851_8721_win,6042,2022-08-19 04:59:05.646,9600,OPT,COM20,06,26 12 34 56 78 00 58 47 00 10 00 00 48 10 00 0...,2022-08-19 04:59:05.946,300.0,ABNT
79,8721,220818_171756_851_8721_win,6042,2022-08-19 04:59:05.950,9600,OPT,COM20,06,26 12 34 56 78 00 59 10 00 00 7F 00 10 00 00 8...,2022-08-19 04:59:06.267,317.0,ABNT
80,8721,220818_171756_851_8721_win,6042,2022-08-19 04:59:06.270,9600,OPT,COM20,06,26 12 34 56 78 00 60 00 00 B6 10 00 00 B7 00 1...,2022-08-19 04:59:06.584,314.0,ABNT
81,8721,220818_171756_851_8721_win,6042,2022-08-19 04:59:06.588,9600,OPT,COM20,06,26 12 34 56 78 10 61 ED 00 10 00 00 EE 10 00 0...,2022-08-19 04:59:06.889,301.0,ABNT


In [310]:
logs['command_name'] = logs.apply(get_command_name, axis=1)
logs

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,timestamp_rx,response_time,command_protocol,command_name
0,8721,220818_171756_851_8721_win,6042,2022-08-19 04:58:14.655,9600,B,COM22,02 01 07 00 09 0C 03,02 01 12 06 01 00 00 00 00 00 00 00 00 00 00 0...,2022-08-19 04:58:14.684,29.0,Production,CALIB_CMD_READ_DATA
1,8721,220818_171756_851_8721_win,6042,2022-08-19 04:58:14.685,9600,B,COM22,02 01 07 00 00 05 03,02 01 86 06 02 00 00 00 01 00 00 00 00 00 00 0...,2022-08-19 04:58:14.844,159.0,Production,CALIB_CMD_READ_DATA
2,8721,220818_171756_851_8721_win,6042,2022-08-19 04:58:14.846,9600,B,COM22,02 01 07 01 08 0C 03,02 01 DE 06 20 00 00 80 01 02 03 00 F2 3F 01 0...,2022-08-19 04:58:15.100,254.0,Production,CALIB_CMD_READ_DATA
3,8721,220818_171756_851_8721_win,6042,2022-08-19 04:58:15.392,9600,B,COM22,02 5D FF 05 00 00 00 00 00 00 00 00 00 00 00 0...,02 5D 06 06 5E 03,2022-08-19 04:58:15.547,155.0,Production,CALIB_CMD_EMUL_MEASURE
4,8721,220818_171756_851_8721_win,6042,2022-08-19 04:58:15.613,9600,B,COM22,02 6A AD 09 00 01 00 10 00 00 00 00 00 00 00 0...,02 6A 06 06 69 03,2022-08-19 04:58:15.739,126.0,Production,CALIB_CMD_EMUL_MEASURE_FRAME_ID9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,8721,220818_171756_851_8721_win,6042,2022-08-19 04:59:05.646,9600,OPT,COM20,06,26 12 34 56 78 00 58 47 00 10 00 00 48 10 00 0...,2022-08-19 04:59:05.946,300.0,ABNT,ABNT_EXT_CMD_06
79,8721,220818_171756_851_8721_win,6042,2022-08-19 04:59:05.950,9600,OPT,COM20,06,26 12 34 56 78 00 59 10 00 00 7F 00 10 00 00 8...,2022-08-19 04:59:06.267,317.0,ABNT,ABNT_EXT_CMD_06
80,8721,220818_171756_851_8721_win,6042,2022-08-19 04:59:06.270,9600,OPT,COM20,06,26 12 34 56 78 00 60 00 00 B6 10 00 00 B7 00 1...,2022-08-19 04:59:06.584,314.0,ABNT,ABNT_EXT_CMD_06
81,8721,220818_171756_851_8721_win,6042,2022-08-19 04:59:06.588,9600,OPT,COM20,06,26 12 34 56 78 10 61 ED 00 10 00 00 EE 10 00 0...,2022-08-19 04:59:06.889,301.0,ABNT,ABNT_EXT_CMD_06


In [311]:
logs.query('command_name == "not found"')

,equipment_type,run_id,test_case,timestamp,baud_rate,slot,serial_port,tx_data,rx_data,timestamp_rx,response_time,command_protocol,command_name
5,8721,220818_171756_851_8721_win,6042,2022-08-19 04:58:15.741,9600,B,COM22,02 6E 11 00 00 00 00 00 00 00 00 00 00 00 00 7...,02 6E 06 06 6D 03,2022-08-19 04:58:15.771,30.0,Production,not found
